In [1]:
import pandas as pd
import numpy as np
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.preprocessing import image
from sklearn.metrics import classification_report
from keras.metrics import Precision
%matplotlib inline

In [2]:
train_dir="./dataset/train"
validate_dir="./dataset/val"
test_dir="./dataset/test"
batch = 20
epochs = 40

In [3]:
def show_img(img,name="image"):
    cv2.imshow(name,img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [4]:
model=Sequential()
model.add(Conv2D(64,(3,3),input_shape=(64,64,1),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(units=128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=36,activation='sigmoid'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=[Precision()])

In [5]:
train_datagen=ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                    horizontal_flip = False)

training_set=train_datagen.flow_from_directory(directory = train_dir,
                                                 target_size = (64, 64),
                                                 color_mode='grayscale',
                                                 batch_size = batch,
                                                 class_mode = 'categorical')

Found 20628 images belonging to 36 classes.


In [6]:
test_datagen=ImageDataGenerator(rescale = 1./255)
test_set=test_datagen.flow_from_directory(directory = test_dir,
                                            target_size = (64, 64),
                                            color_mode='grayscale',
                                            batch_size = batch,
                                            class_mode = 'categorical')

Found 972 images belonging to 36 classes.


In [7]:
val_datagen=ImageDataGenerator(rescale = 1./255)
val_set=val_datagen.flow_from_directory(directory = validate_dir,
                                            target_size = (64, 64),
                                            color_mode='grayscale',
                                            batch_size = batch,
                                            class_mode = 'categorical')

Found 2160 images belonging to 36 classes.


In [8]:
model.fit_generator(
    training_set,
    steps_per_epoch=training_set.samples // batch,
    epochs=epochs,
    validation_data=val_set,
    validation_steps=val_set.samples // batch)

Epoch 1/40


C:\Users\Akash\AppData\Local\Temp\ipykernel_24408\3616471183.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


1031/1031 [==============================] - 67s 64ms/step - loss: 2.0390 - precision: 0.1074 - val_loss: 0.5008 - val_precision: 0.2384
Epoch 2/40
1031/1031 [==============================] - 69s 67ms/step - loss: 0.8588 - precision: 0.2222 - val_loss: 0.1735 - val_precision: 0.2605
Epoch 3/40
1031/1031 [==============================] - 73s 71ms/step - loss: 0.6074 - precision: 0.2414 - val_loss: 0.1096 - val_precision: 0.2940
Epoch 4/40
1031/1031 [==============================] - 59s 57ms/step - loss: 0.5127 - precision: 0.2557 - val_loss: 0.1165 - val_precision: 0.3137
Epoch 5/40
1031/1031 [==============================] - 55s 54ms/step - loss: 0.4418 - precision: 0.2678 - val_loss: 0.0836 - val_precision: 0.3229
Epoch 6/40
1031/1031 [==============================] - 58s 56ms/step - loss: 0.3997 - precision: 0.2772 - val_loss: 0.0778 - val_precision: 0.3342
Epoch 7/40
1031/1031 [==============================] - 56s 54ms/step - loss: 0.3681 - precision: 0.2892 - val_loss: 0.0620

In [9]:
predict = model.predict_generator(test_set,steps = test_set.samples//batch)
y_pred = np.argmax(predict, axis=-1)
test_labels=test_set.classes 
print(classification_report(test_labels[:len(y_pred)], y_pred))

C:\Users\Akash\AppData\Local\Temp\ipykernel_24408\664459255.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predict = model.predict_generator(test_set,steps = test_set.samples//batch)


              precision    recall  f1-score   support

           0       0.04      0.04      0.04        27
           1       0.04      0.04      0.04        27
           2       0.04      0.04      0.04        27
           3       0.04      0.04      0.04        27
           4       0.00      0.00      0.00        27
           5       0.00      0.00      0.00        27
           6       0.00      0.00      0.00        27
           7       0.04      0.04      0.04        27
           8       0.00      0.00      0.00        27
           9       0.00      0.00      0.00        27
          10       0.04      0.04      0.04        27
          11       0.00      0.00      0.00        27
          12       0.11      0.11      0.11        27
          13       0.00      0.00      0.00        27
          14       0.00      0.00      0.00        27
          15       0.00      0.00      0.00        27
          16       0.00      0.00      0.00        27
          17       0.07    

In [20]:
model.save('modelwts2.h5')